<a href="https://colab.research.google.com/github/DmitrijLeontev/Dmitrij_Leontev/blob/main/Prototype8_Dantist_curses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Подтягивание базы знаний с Github,

вмонтирован  промптsistem,

организован подход с обращением к ключю OpenAI и токену Githab через секретные ключи Google collab,

подсчет токенов для каждого фрагмента и построение графика,

суммаризация диалога,

отображение стоимости вопрос + промпт + ответ и

суммы за весь диалог.

для выхода из режима диалога с консультантом наберите  в окне диалоговом - stop



Наша задача учесть еще и промпт.

Из документации OpenAI берем прайс на токены:

стоимость 1M input tokens: $0.15(входящие токены)/1Млн- вопрос

стоимость 1M input tokens: $0.15(входящие токены)/1Млн- промпт и  ближайшие 6 подтянутых чанков

стоимость 1M output tokens: $0.6(исходящие токены)/1Млн- ответ.

Консультант получает вопрос от пользователя и одновременно получает промпт sistem как инструкцию и на основе этого формирует ответ.

1. Получается, что мы должны учесть цену за вопрос(input tokens)+ цену за токены  промпта(input tokens):
Сумма за вопрос и промт  =($0.15/1000000)* n токенов вопроса = + ($0.15/1000000)* n токенов 6 подтянутых чанков = $0.00000015* n токенов вопроса+$0.00000015* n  токенов промпта + $0.00000015* n токенов подтянутых 6 ближайших чанков

2. Ответ идет уже без промпта, так как уже при подаче вопроса происходит обучение по промпту. Консультант принял вопрос и промпт и отвечает. В ответе считается число токенов ответа :

Цена в долларах за токен ответа: $0.6/1000000= $0.0000006

3. Учитывая пункты 1. и 2. мы получаем сумму за сессию вопрос, ответ, промпт:

($0.15/1000000)* n токенов вопроса + ($0.15/1000000)* n токенов промпта систем + ($0.15/1000000)* n токенов 6 ближайших подтянутых чанков+ ($0.6/1000000)* n токенов ответа.

n - количество токенов

4. В конце надо сложить все суммы за все сессии вопрос+ промпт+ответ+цена подтянутых 6 чанков, ближайших, в каждой сессии и мы получим сумму за весь диалог.

Приступим.

In [ ]:
!pip install -q langchain_openai==0.0.2 faiss-cpu==1.7.4 openai==1.6.1 tiktoken==0.5.2 langchain_community==0.0.11 langchain==0.1.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 500.5 kB/s eta 0:00:00


In [ ]:
!pip install PyGithub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.6/362.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 21.3 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
import os
from google.colab import userdata
import re
import requests
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from github import Github
import tiktoken
import matplotlib.pyplot as plt



In [ ]:
# Установка API ключа OpenAI из секретных данных Colab
openai_api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_api_key

# Создание клиента OpenAI
client = OpenAI()

# Установка GitHub API ключа
github_token = userdata.get('GITHUB_TOKEN')

Добавил расчет стоимости диалога >>>

In [ ]:
# Средняя стоимость токенов
INPUT_TOKEN_COST = 0.15 / 1000000 # USD за 1000000 входящих токенов
OUTPUT_TOKEN_COST = 0.6 / 1000000  # USD за 1000000 исходящих токенов

def calculate_cost(question_token_count: int, prompt_token_count: int, answer_token_count: int) -> float:
    """
    Функция для расчета стоимости на основе количества токенов.
    """
    question_cost = (question_token_count + prompt_token_count) * INPUT_TOKEN_COST
    answer_cost = answer_token_count * OUTPUT_TOKEN_COST
    return question_cost + answer_cost


In [ ]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

Блок отвечающий за подтягивание базы знаний с Github

1) нужно на место многоточия вставить свой токен Github

2) Загрузка данных из GitHub: Функция load_document_from_github загружает текстовый файл из GitHub, используя токен доступа.

Проверка данных: Проверяется, удалось ли загрузить данные, и выводятся первые 1000 символов для проверки.

Обработка текста: Текст разбивается на чанки с помощью MarkdownHeaderTextSplitter.

Подсчет токенов: Функция num_tokens_from_string подсчитывает количество токенов в каждом чанке.

Построение графика: Строится гистограмма распределения длин чанков в токенах.

In [ ]:
# Получение токена из секретных данных Colab
github_token = userdata.get('GITHUB_TOKEN')

# Проверка на наличие токена
if not github_token:
    raise ValueError("GitHub token is missing. Please set it in the Colab secrets.")

# Загрузка документа с GitHub
def load_document_from_github(url: str, github_token: str) -> str:
    match = re.match(r"https://github.com/([^/]+)/([^/]+)/blob/([^/]+)/(.*)", url)
    if not match:
        raise ValueError("Invalid GitHub URL")
    username, repo_name, branch, file_path = match.groups()

    g = Github(github_token)
    repo = g.get_repo(f"{username}/{repo_name}")
    file_content = repo.get_contents(file_path, ref=branch)
    return file_content.decoded_content.decode('utf-8')

# Загрузка базы знаний из GitHub
github_url = 'https://github.com/NeuronsUII....' # здесь заменить на ссылку на базу знаний с Github

try:
    data_from_github = load_document_from_github(github_url, github_token)
    print(data_from_github[:1000])  # Вывод первых 1000 символов документа для проверки
except Exception as e:
    print(f"Error loading document: {e}")

# Убедимся, что текст был загружен
if 'data_from_github' in locals():
    # Разделение текста на чанки
    splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[("#", "Header 1")])
    fragments = splitter.split_text(data_from_github)

    # Функция для подсчета токенов
    def num_tokens_from_string(text: str, model_name: str) -> int:
        encoding = tiktoken.get_encoding(model_name)
        tokens = encoding.encode(text)
        return len(tokens)

    # Проверка, что каждый фрагмент является строкой
    fragments = [fragment if isinstance(fragment, str) else str(fragment) for fragment in fragments]

    fragment_token_counts = [num_tokens_from_string(fragment, "cl100k_base") for fragment in fragments]

    # Построение графика распределения длин токенов
    plt.hist(fragment_token_counts, bins=50, alpha=0.5, label='Fragments')
    plt.title('Распределение длин чанков в токенах')
    plt.xlabel('Token Count')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()
else:
    print("Ошибка: данные не были загружены из GitHub.")



бдлок загрузки базы знаний с Github закончен

In [ ]:
# Вывод первых 1000 символов документа для проверки
print(data_from_github[:1000])

In [ ]:
# Указание заголовков, по которым будет происходить разделение
headers_to_split_on = [
    ("#", "Header 1"),
]

# Создание объекта MarkdownHeaderTextSplitter для разделения текста
splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# Разделение текста на чанки
chunks = splitter.split_text(data_from_github)

# Вывод чанков
for i, chunk in enumerate(chunks):
    print(f"Чанк {i+1}:\n{chunk}\n")

In [ ]:
# Инструкция в system

system = load_document_text('https://docs.google.com/document/d/1EEqTk........') # заполняем параметр ссылкой на составленный промпт. Не забываем открыть доступ

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Вывод первых 4000 символов system
print(system[:4000])

In [ ]:
# выводим несколько чанков, чтобы убедиться, что все получилось в необходимом нам формате
chunks[0:5]

In [ ]:
len(chunks)

17

In [ ]:
# Инициализируем модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(chunks, embeddings)

Отдельный блок для сохранения базы Faiss (если необходимо)

In [ ]:
# Задаем имя и путь для сохранения файла
folder_path = '/content/drive/MyDrive/'
index_name = "db"

In [ ]:
# Сохраняем db на ваш Google Drive
db.save_local(folder_path=folder_path, index_name=index_name)

In [ ]:
# Путь к сохраненному файлу
saved_file_path = f"{folder_path}{index_name}"

In [ ]:
# Настройка общего доступа и получение ссылки вручную
print(f"Файл сохранен в {saved_file_path}. Перейдите на Google Drive, найдите файл и настройте общий доступ.")

Файл сохранен в /content/drive/MyDrive/db. Перейдите на Google Drive, найдите файл и настройте общий доступ.


In [ ]:
# Вывод ссылки для ручного копирования и настройки общего доступа
drive_link = f"https://drive.google.com/drive/u/0/my-drive"
print(f"Ссылка на ваш Google Drive: {drive_link}")

Конец блока сохранения базы Faiss

In [ ]:
def insert_newlines(text: str, max_len: int = 170) -> str:
    """
    Функция разбивает длинный текст на строки определенной максимальной длины.
    """
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

In [ ]:
def answer_index(system, topic, search_index, verbose=1):
    """
    Функция для ответа на вопросы с использованием индекса и расчета стоимости.

    Args:
      system: Промпт системы
      topic: Вопрос пользователя
      search_index: Индекс базы знаний
      verbose: Флаг для вывода отладочной информации

    Returns:
      answer: Ответ на вопрос
      cost: Общая стоимость в USD
      question_token_count: Количество токенов вопроса
      prompt_token_count: Количество токенов промпта
      answer_token_count: Количество токенов ответа
    """
    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=6)
    if verbose: print('\n ===========================================: ')

    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)

    client = OpenAI()
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос. Документ с информацией для ответа: {message_content}\n\nВопрос пользователя: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')

    completion = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=messages,
        temperature=0
    )

    answer = completion.choices[0].message.content

    # Подсчет токенов
    question_token_count = num_tokens_from_string(topic, "cl100k_base")
    prompt_token_count = num_tokens_from_string(system, "cl100k_base")
    answer_token_count = num_tokens_from_string(answer, "cl100k_base")
    cost = calculate_cost(question_token_count, prompt_token_count, answer_token_count)

    return answer, cost, question_token_count, prompt_token_count, answer_token_count


In [ ]:
def summarize_questions(dialog):
    """
    Функция возвращает саммаризированный текст диалога.
    """
    messages = [
        {"role": "system", "content": "Ты - нейро-саммаризатор. Твоя задача - саммаризировать диалог, который тебе пришел. Если пользователь назвал свое имя, обязательно отрази его в саммаризированном диалоге"},
        {"role": "user", "content": "Саммаризируй следующий диалог консультанта и пользователя: " + " ".join(dialog)}
    ]

    completion = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=messages,
        temperature=0,
    )

    return completion.choices[0].message.content


In [ ]:
def answer_user_question_dialog(system, db, user_question, question_history):
    """
    Функция возвращает ответ на вопрос пользователя с расчетом стоимости.
    """
    summarized_history = ""
    if len(question_history) > 0:
        # Извлекаем только вопросы и ответы для саммаризации
        summarized_history = "Вот саммаризированный предыдущий диалог с пользователем: " + \
                              summarize_questions([q + ' ' + (a if a else '') for q, a, *_ in question_history])

    topic = summarized_history + " Актуальный вопрос пользователя: " + user_question

    # Получаем ответ и стоимость
    answer, cost, question_token_count, prompt_token_count, answer_token_count = answer_index(system, topic, db)

    question_history.append((user_question, answer if answer else '', cost, question_token_count, prompt_token_count, answer_token_count))

    # Выводим саммаризированный текст, который видит модель
    if summarized_history:
        print('****************************')
        print(summarized_history)
        print('****************************')

    print(f"Стоимость текущего вопроса и ответа: ${cost:.4f}")
    print(f"Количество токенов - Вопрос: {question_token_count}, Промпт: {prompt_token_count}, Ответ: {answer_token_count}")

    return answer, cost


In [ ]:
def run_dialog(system, db):
    """
    Функция запускает диалог между пользователем и нейро-консультантом.
    """
    question_history = []
    total_cost = 0.0

    while True:
        user_question = input('Пользователь: ')
        if user_question.lower() == 'stop':
            break
        answer, cost = answer_user_question_dialog(system, db, user_question, question_history)
        total_cost += cost
        print('Консультант:', insert_newlines(answer))

    print(f"Общая стоимость диалога: ${total_cost:.2f}")
    return


In [ ]:
# Провеям работу нейро-консультанта на нескольких самостоятельно сгенерированных вопросах.
# После запуска, вводите ваши вопросы в консоли. Для завершения сеанса диалога, введите stop.
run_dialog(system, db)

Вывод:

консультант работает и считает каждый вопрос - ответ с учетом промпта подаваемого с вопросом. Также консультант считает по итогу сумму за весь диалог. Контекст на моих вопросах держит хорошо. Суммаризацию диалога показывает.

Следующий шаг: Создание модели(функции) , которая будет сохранять ответы и вопросы и подтянутые чанки в отдельную базу знаний, затем при поступлении вопроса будет искать ответ в этой базе знаний. Если ответ не найден , то она ищет ответ в основной базе знаний. И  так продолжается пока диалог не закончится.